In [1]:
!pip show tenseal

Name: tenseal
Version: 0.3.16
Summary: A Library for Homomorphic Encryption Operations on Tensors
Home-page: https://github.com/OpenMined/TenSEAL
Author: OpenMined
Author-email: info@openmined.org
License: Apache-2.0
Location: c:\Developments\Backend\Python\test offline jupyter\venv\Lib\site-packages
Requires: 
Required-by: 


In [ ]:
import numpy as np

def load_quantized_weights(file_path, scale=2**20):
    weights = np.fromfile(file_path, dtype=np.float32)

    quantized_weights = np.round(weights * scale).astype(np.int64)
    return quantized_weights

scale = 2**40
conv1_weights = load_quantized_weights("./chest_xray/trainable_weights/conv1_weights.bin", scale)
conv2_weights = load_quantized_weights("./chest_xray/trainable_weights/conv2_weights.bin", scale)
conv3_weights = load_quantized_weights("./chest_xray/trainable_weights/conv3_weights.bin", scale)
dense_weights = load_quantized_weights("./chest_xray/trainable_weights/dense_weights.bin", scale)

In [ ]:

import tenseal as ts

# This is the Setup for TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=32768,
    coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 60]
)

# We are Setting global scale
context.global_scale = scale

# Let us Generate keys
context.generate_galois_keys()

print("TenSEAL context have been created.")

TenSEAL context have been created.


In [4]:
import numpy as np
from PIL import Image

def preprocess_and_encrypt_batched(image_path, ctx, scale=2**20, slot_count=16384):
    # Load image
    img = Image.open(image_path).convert('RGB').resize((150, 150))
    img_array = np.array(img).astype(np.float32) / 255.0
    flat_input = img_array.flatten()

    # Quantize to fixed-point integers
    quantized = np.round(flat_input * scale).astype(np.int64)

    # Split input into batches based on slot count
    batches = [quantized[i:i + slot_count] for i in range(0, len(quantized), slot_count)]

    # Encrypt each batch
    encrypted_batches = [ts.ckks_vector(ctx, batch) for batch in batches]

    return encrypted_batches, img_array.shape

# Example usage
encrypted_input, input_shape = preprocess_and_encrypt_batched("./chest_xray/test/NORMAL/IM-0001-0001.jpeg", context)

In [ ]:
import numpy as np
import tenseal as ts

def fhe_conv2d_simulated(batch, kernel, kernel_size=3, channels=3):
    # 1. Get info about encrypted vector
    batch_size = batch.size()
    window_size = kernel_size * kernel_size * channels  # e.g., 3x3x3 = 27

    # 2. Ensure kernel is a flat numpy array
    if isinstance(kernel, list):
        kernel = np.array(kernel)  # Convert list to numpy array if needed
    kernel_flat = kernel.flatten()[:window_size]  # Take only first 27 elements

    # 3. Create padded kernel to match batch size
    kernel_padded = np.zeros(batch_size)
    kernel_padded[:window_size] = kernel_flat  # Now this should work

    # 4. Convert to plaintext tensor
    kernel_pt = ts.plain_tensor(kernel_padded)

    # 5. Apply dot product with encrypted input
    result = batch.dot(kernel_pt)

    return result

In [24]:
def fhe_conv2d(batches, weights, kernel_size=3, in_channels=3, out_channels=32):
    results = []
    for batch in batches:
        res = fhe_conv2d_simulated(batch, weights, kernel_size, in_channels)
        results.append(res)
    return results

In [ ]:
def fhe_relu(x_list):
    results = []
    for vec in x_list:
        squared = vec.mul(vec)
        results.append(squared)
    return results

In [8]:
def fhe_max_pooling(x_list):
    """Approximates max pooling using average"""
    return [vec.sum().mul(1.0 / vec.size()) for vec in x_list]

In [9]:
def fhe_dense_layer(x_batches, weights):
    result = []
    for batch in x_batches:
        print(batch.size())
        w_plain = ts.plain_tensor(weights[:batch.size()])
        result.append(batch.dot(w_plain))
    return ts.ckks_vector_sum(result)

In [ ]:
def encrypted_predict_batched(encrypted_batches):
    # Only ONE Conv + ReLU + MaxPooling
    x = fhe_conv2d(encrypted_batches, conv1_weights)
    x = fhe_relu(x)
    x = fhe_max_pooling(x)

    # Flatten and combine results before dense layer
    if len(x) == 0:
        raise ValueError("No values after max pooling")

    x_combined = sum(x[1:], x[0])  # Combine pooled outputs

    # Truncate or pad input to match dense layer size
    desired_size = dense_weights.shape[0]
    actual_size = x_combined.size()

    if desired_size > actual_size:
        # Pad with zeros
        padding = np.zeros(desired_size)
        padding[:actual_size] = x_combined.decrypt()
        padded_input = ts.ckks_vector(x_combined.context(), padding)
        masked_input = padded_input
    else:
        # Truncate input
        mask = np.zeros(actual_size)
        mask[:desired_size] = 1.0
        mask_pt = ts.plain_tensor(mask)
        masked_input = x_combined.mul(mask_pt)

    # Now do dot product
    output = masked_input.dot(ts.plain_tensor(dense_weights))
    return output


# Run prediction
encrypted_output = encrypted_predict_batched(encrypted_input)

In [ ]:
# Run prediction
encrypted_output = encrypted_predict_batched(encrypted_input)

# Step 1: Decrypt
decrypted_output = encrypted_output.decrypt()

# Step 2: Rescale
predicted_logit = decrypted_output[0] / (1 << 20)

# Step 3: Apply sigmoid
import math
sigmoid = lambda x: 1 / (1 + math.exp(-x))
predicted_prob = sigmoid(predicted_logit)

# Step 4: Print result
print(f"Predicted logit: {predicted_logit}")
print(f"Pneumonia probability: {predicted_prob:.4f}")

# Step 5: Make a prediction
if predicted_prob > 0.5:
    print("Prediction: Pneumonia detected")
else:
    print("Prediction: No pneumonia")